# Quick Start

In [ ]:
# this cell's metadata contains
# "nbsphinx": "hidden" so it is hidden by nbsphinx
PATH_YIELD_ENGINE = 'src'
def set_paths() -> None:
    import sys
    import os
    
    if 'cwd' not in globals():
        global cwd
        cwd = os.path.join(os.getcwd(), os.pardir, os.pardir, os.pardir)
        os.chdir(cwd)   
    print(f"working dir is '{os.getcwd()}'")                            
    if PATH_YIELD_ENGINE not in sys.path:
        sys.path.insert(0, PATH_YIELD_ENGINE)
    print(f"added `{sys.path[0]}` to python paths")
set_paths()



## TL;DR

We first make our imports:

In [ ]:
import pandas as pd
from sklearn import datasets
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

from gamma import Sample
from gamma.viz.dendrogram import DendrogramDrawer, DendrogramFeatMapStyle, DendrogramLineStyle, DendrogramReportStyle
from gamma.sklearndf.pipeline import PipelineDF, ModelPipelineDF
from gamma.sklearndf.transformation import SimpleImputerDF, MissingIndicatorDF, StandardScalerDF
from gamma.sklearndf.transformation.extra import BorutaDF
from gamma.sklearndf.regression import RandomForestRegressorDF, LGBMRegressorDF
from gamma.model.inspection import ModelInspector
from gamma.model.prediction import PredictorFitCV
from gamma.model.selection import ModelGrid, ModelRanker, summary_report
from gamma.model.validation import CircularCrossValidator
from gamma.yieldengine.simulation import UnivariateSimulator
from gamma.yieldengine.partition import ContinuousRangePartitioning
from gamma.yieldengine.viz import SimulationDrawer, SimulationPlotStyle

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

We load the Boston dataset, a regression problem on house prices in boston suburbs:

In [ ]:
boston = datasets.load_boston()
TARGET='MEDIAN_HOUSE_PRICE'
df = pd.DataFrame(data=boston.data, columns=boston.feature_names).assign(MEDIAN_HOUSE_PRICE=boston.target)
df.head()

In [ ]:
sample = Sample(observations=df, target_name=TARGET)

In [ ]:
sample.features.head()

In [ ]:
sample.target.head()

##  Dataframes instead of arrays

In [ ]:
imputed_array = SimpleImputer().fit_transform(sample.features)
imputed_array

In [ ]:
imputed_df = SimpleImputerDF().fit_transform(sample.features)
imputed_df.head()

In [ ]:
sample_imputed = SimpleImputerDF().fit_transform_sample(sample=sample)

## Cross-validation and model selection

In [ ]:
preprocessing = PipelineDF(steps=[('impute', SimpleImputerDF()), ('normalize', StandardScalerDF())])

In [ ]:
rf_pipeline = ModelPipelineDF(
    predictor=RandomForestRegressorDF(random_state=0), preprocessing=preprocessing)
lgbm_pipeline = ModelPipelineDF(
    predictor=LGBMRegressorDF(random_state=0), preprocessing=preprocessing)

In [ ]:
grids = [
    ModelGrid(pipeline=rf_pipeline, predictor_parameters={"n_estimators": [10, 50]}),
    ModelGrid(pipeline=lgbm_pipeline, predictor_parameters={"learning_rate": [0.1, 0.2]})
]

In [ ]:
cv = CircularCrossValidator(test_ratio=1/3, num_splits=3)
ranker = ModelRanker(grids=grids, cv=cv)
ranking = ranker.run(sample_imputed, n_jobs=-3)
print(summary_report(ranking))

## Model inspection

In [ ]:
top_model = ranking[0].model

In [ ]:
predictor = PredictorFitCV(model=top_model, cv=cv, sample=sample_imputed)

We can retrieve the predictions for all splits, hence make a full inspection of the model:

In [ ]:
predictor.predictions_for_split(split_id=0).head()

In [ ]:
inspector = ModelInspector(predictor)
inspector.shap_matrix().head()

In [ ]:
inspector.feature_importances().head()

In [ ]:
linkage_tree = inspector.cluster_dependent_features()
ax = plt.figure(figsize=(10, 10)).add_subplot(111)
style_matplotlib = DendrogramFeatMapStyle(ax)
DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style_matplotlib).draw()

In [ ]:
style_report = DendrogramReportStyle()
DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style_report).draw()

## Feature independence engineering

In [ ]:
sample_new = sample_imputed.select_features(['RM', 'LSTAT','AGE', 'CRIM'])

And we can rerun our pipeline to select, fit and inspect our model with this new data:

In [ ]:
ranking = ranker.run(sample_new, n_jobs=-3)
top_model = ranking[0].model
predictor = PredictorFitCV(model=top_model, cv=cv, sample=sample_new)
inspector = ModelInspector(predictor)
linkage_tree = inspector.cluster_dependent_features()
ax = plt.figure(figsize=(10, 10)).add_subplot(111)
style = DendrogramFeatMapStyle(ax)
DendrogramDrawer(title=TARGET, linkage_tree=linkage_tree, style=style).draw()

## Simulation

In [ ]:
simulator = UnivariateSimulator(model_fit=predictor)

In [ ]:
partition = ContinuousRangePartitioning(values=predictor.sample.features["LSTAT"])

In [ ]:
univariate_simulation = simulator.simulate_feature(feature_name="LSTAT", partitioning=partition)

In [ ]:
ax = plt.figure(figsize=(10,10)).add_subplot(111)
style = SimulationPlotStyle(ax)
drawer = SimulationDrawer(simulation=univariate_simulation, style=style)
drawer.draw()